In [0]:
import spacy
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wnet
nlp = spacy.load('en')
import os


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Read data
The input dataset is split into two files:
- train.nl- It has the input natural language queries.
- trainsql- It has the SQL query corresponding to the Natural language query in train.nl.

In [0]:
#os.path.isfile("train.nl")
enlines = open('train.nl', encoding='utf-8').read().split('\n')
sqllines = open('train.sql', encoding='utf-8').read().split('\n')

In [0]:
enlines=enlines[:len(enlines)-1]
sqllines=sqllines[:len(sqllines)-1]

In [0]:
print(enlines[-1])
print(sqllines[-1])
print(len(enlines))
print(len(sqllines))

i want a flight from pittsburgh to los angeles that departs after 6pm on thursday
SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'PITTSBURGH' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'LOS ANGELES' AND ( flight_1.flight_days = days_1.days_code AND days_1.day_name = date_day_1.day_name AND date_day_1.year = 1991 AND date_day_1.month_number = 5 AND date_day_1.day_number = 24 AND flight_1.departure_time > 1800 ) )
4379
4379


# Paraphrase

The first word of a sentence is always paraphrased. If it is not the first word we do the following:
- If it is a noun we do not paraphrase it. The nouns could be potential candidates for becoming a database object- namely table name, column name etc. Hence we avoid losing it in order to accomodate it in our output sequence.
- There is no gain in paraphrasing stop words, hence we exclude that as well
- If the dependency parse tree yields the tag nobj,pobj or dobj it is not paraphrased
- If the dependency parse tree yields the tag compound or xcomp then we exclude it in order to ensure that we do not paraphrase part of a phrase or word.
- If the word clears all the above conditions, then we use the word along with its pos tag to obtain its synsets from wordnet. We use the first three synsets only. We make sure that each paraphrased word occurs in combination with each of the other for a given sentence.

In [0]:
def paraphrase(enlines):
  new_enlines=[]
  new_sqllines=[]
  syn_dict={}
  for sindex in range(len(enlines)):
    spline=nlp(enlines[sindex])
    comb_sent=[enlines[sindex]] 
    sql_comb=[sqllines[sindex]]
    for i in range(len(spline)):
      token=spline[i]      
      word=str(token)
      if i==0 or (not(token.is_stop) and token.pos_!='NOUN' and token.dep_ not in ('nobj','dobj','compound','xcomp','pobj')):     
        if '{},{}'.format(word,token.pos_) not in syn_dict:          
          list_syn=[]
          if token.pos_=='ADJ':
            list_syn=wnet.synsets(word,wnet.ADJ)
          elif token.pos_=='ADV':
            list_syn=wnet.synsets(word,wnet.ADV)
          else:
            list_syn=wnet.synsets(word,wnet.VERB)
          if len(list_syn)>0:
            temp=[each.lemmas()[0].name() for each in list_syn[1:3]]
            temp.extend([lem.name() for lem in list_syn[0].lemmas()[0:3]])
            temp=list(dict.fromkeys(temp))
            temp=list(set(temp)-set([word])-set([token.lemma_]))
            if len(temp)>0:              
              syn_dict['{},{}'.format(word,token.pos_)]=temp
        if '{},{}'.format(word,token.pos_) in syn_dict:
          initial=comb_sent[:]
          for sent in initial:
            for syntext in syn_dict['{},{}'.format(word,token.pos_)]:
              sentlist=sent.split(" ")
              comb_sent.append(" ".join(sentlist[:i]+[syntext]+sentlist[i+1:]))                
    sql_comb=sql_comb*len(comb_sent)            
    new_enlines.extend(comb_sent)
    new_sqllines.extend(sql_comb)
  return(syn_dict,new_enlines,new_sqllines)


In [0]:
syn_dict,para_enlines,para_sqllines=paraphrase(enlines)

In [0]:
print(para_enlines[0:10])
print(len(para_enlines))
print(para_sqllines[0:10])
print(len(para_sqllines))
print(len(list(syn_dict.keys())))
print(syn_dict)

['list all the flights that arrive at general mitchell international from various cities', 'name all the flights that arrive at general mitchell international from various cities', 'list all the flights that get at general mitchell international from various cities', 'list all the flights that come at general mitchell international from various cities', 'name all the flights that get at general mitchell international from various cities', 'name all the flights that come at general mitchell international from various cities', 'give me the flights leaving denver august ninth coming back to boston', 'yield me the flights leaving denver august ninth coming back to boston', 'give me the flights go_forth denver august ninth coming back to boston', 'give me the flights go_away denver august ninth coming back to boston']
41222
["SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.air

# Write File
After paraphrasing the output is written to the files new_train.nl and new_train.sql.

In [0]:
X_data='\n'.join(para_enlines)
Y_data='\n'.join(para_sqllines)

In [0]:
with open("new_train.nl",'w') as f:
  f.write(X_data)
with open("new_train.sql",'w') as f:
  f.write(Y_data)